In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import copy 
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tff.backends.reference.set_reference_context()

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import models

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from operator import add

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
origTra = open('../Datasets/pendigits/pendigits-orig.tra', 'r').read()

In [5]:
listOrigTra = []
for i in origTra.split('COMMENT ')[1:]:
    listOrigTra.append(i.split()[1])
    
len(listOrigTra)

7494

In [6]:
tra = open('../Datasets/pendigits/pendigits.tra', 'r').read()
listTra = []
for i in tra.split('\n'):
    listTra.append(i[-1])
    
len(listTra)

7494

In [7]:
origTes = open('../Datasets/pendigits/pendigits-orig.tes', 'r').read()

In [8]:
listOrigTes = []
for i in origTes.split('COMMENT ')[1:]:
    listOrigTes.append(i.split()[1])
    
len(listOrigTes)

3498

In [9]:
listOrigTes = [str(int(i)+30) for i in listOrigTes]

In [10]:
tes = open('../Datasets/pendigits/pendigits.tes', 'r').read()
listTes = []
for i in tes.split('\n'):
    listTes.append(i[-1])
    
len(listTes)

3498

In [11]:
columns = ['Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6', 'Input7', 'Input8', 'Input9', 'Input10', 'Input11', 'Input12', 'Input13', 'Input14', 'Input15', 'Input16', 'digit']

In [12]:
df_tra = pd.read_csv('../Datasets/pendigits/pendigits.tra', delimiter = ',', names=columns , header=None)
df_tra.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [13]:
df_tra_w = df_tra.copy()
df_tra_w['writer'] = listOrigTra
df_tra_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [14]:
df_tra_w.to_csv('tra.csv')

In [15]:
df_tes = pd.read_csv('../Datasets/pendigits/pendigits.tes', delimiter = ',', names=columns , header=None)
df_tes.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9


In [16]:
df_tes.to_csv('TEST.csv')

In [17]:
df_tes_w = df_tes.copy()
df_tes_w['writer'] = listOrigTes
df_tes_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8,36
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8,42
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8,35
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9,37
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9,35


In [18]:
df = pd.concat([df_tra_w,df_tes_w])
df.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [19]:
d0 = df_tra_w.loc[df_tra_w.digit==0].shape[0]
d1 = df_tra_w.loc[df_tra_w.digit==1].shape[0]
d2 = df_tra_w.loc[df_tra_w.digit==2].shape[0]
d3 = df_tra_w.loc[df_tra_w.digit==3].shape[0]
d4 = df_tra_w.loc[df_tra_w.digit==4].shape[0]
d5 = df_tra_w.loc[df_tra_w.digit==5].shape[0]
d6 = df_tra_w.loc[df_tra_w.digit==6].shape[0]
d7 = df_tra_w.loc[df_tra_w.digit==7].shape[0]
d8 = df_tra_w.loc[df_tra_w.digit==8].shape[0]
d9 = df_tra_w.loc[df_tra_w.digit==9].shape[0]

print(d0)
print(d1)
print(d2)
print(d3)
print(d4)
print(d5)
print(d6)
print(d7)
print(d8)
print(d9)

D={
        0:d0,
        1:d1,
        2:d2,
        3:d3,
        4:d4,
        5:d5,
        6:d6,
        7:d7,
        8:d8,
        9:d9,
    
}

780
779
780
719
780
720
720
778
719
719


In [20]:
pickle.dump(D,open('COUNT','wb'))

In [21]:
def getClients(data):
    arr = []
    no_of_writers = len(set(data['writer']))
    for i in range(no_of_writers):
        arr.append([])
    for index, row in data.iterrows():
        rowA = row.to_numpy()
        writer = int(rowA[-1])
        data = rowA[:17]
        arr[writer-1].append(data)
    return arr

In [22]:
def get_batches(HOLDER):
    federated_data = []
    for i in HOLDER:
        client = []
        for index in range(len(i)//20):
            X= []
            Y= []
            for elements in i[index*20:index*20+20]:
                X.append(np.array([np.float32(elements[0]),
                                   np.float32(elements[1]),
                                   np.float32(elements[2]),
                                   np.float32(elements[3]),
                                   np.float32(elements[4]),
                                   np.float32(elements[5]),
                                   np.float32(elements[6]),
                                   np.float32(elements[7]),
                                   np.float32(elements[8]),
                                   np.float32(elements[9]),
                                   np.float32(elements[10]),
                                   np.float32(elements[11]),
                                   np.float32(elements[12]),
                                   np.float32(elements[13]),
                                   np.float32(elements[14]),
                                   np.float32(elements[15])]))
                Y.append(np.array(np.int32(elements[-1])))
            client.append({
                'x':np.array(X),
                'y':np.array(Y)
            })
        federated_data.append(client)
    return federated_data

In [23]:
# federated_train_data = getClients(df_tra_w)
# federated_test_data = getClients(df_tes_w)

In [24]:
# federated_train_data =  get_batches(train)
# federated_validation_data =  get_batches(validation)
# federated_test_data =  get_batches(test)

In [25]:
def getModel():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(10,activation="sigmoid")
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [26]:
import numpy as np
from collections import Counter

def getweight(df):
    LBL = Counter(df.digit)
    maen_arr = np.mean(list(map(lambda x:x[1]/D[x[0]],list(LBL.items()))))
    return maen_arr

In [27]:
writer_data = df_tra_w.groupby(df_tra_w.writer)

In [28]:
import pickle
from tqdm.notebook import tqdm
GOD = []

In [29]:
d ={}
for i in range(1,31):
    d[i]=writer_data.get_group('{}'.format(i)).sample(50)
    
pickle.dump(d,open('R1','wb'))

X = []
for i in range(20):
    d = {}
    for i in range(1,31):
        d[i]=writer_data.get_group('{}'.format(i)).sample(50)
    X.append(d)

pickle.dump(X,open('Rs','wb'))

In [30]:
for i in tqdm(range(1)):
    d ={}
    for i in range(1,31):
        d[i]=writer_data.get_group('{}'.format(i)).sample(50)

#     pickle.dump(d,open('R1','wb'))

    X = []
    for i in range(10):
        d = {}
        for i in range(1,31):
            d[i]=writer_data.get_group('{}'.format(i)).sample(50)
        X.append(d)
        
    GOD.append({
        'fs':d,
        'snd':X
    })

In [31]:
pickle.dump(GOD,open('GOD.pkl','wb'))

In [32]:
df_sample_test = pd.concat([df_tra, df_tes], axis=0).sample(25)
pickle.dump(df_sample_test, open('df_sample_test','wb'))

In [33]:
df_sample_test

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
1894,81,98,26,93,59,67,95,100,88,66,100,24,48,0,0,14,9
3390,0,93,52,100,94,87,80,63,53,42,23,20,42,3,100,0,2
4425,0,100,1,58,20,17,58,0,94,19,100,64,69,92,28,90,0
5644,100,85,67,100,56,66,96,79,90,65,86,28,54,0,0,11,9
1432,0,94,69,100,97,69,72,34,49,0,90,32,100,50,19,49,7
1863,100,94,29,100,0,72,70,79,88,98,64,65,50,33,43,0,9
3460,83,89,36,100,45,59,92,84,92,75,100,28,58,0,0,19,9
1488,55,100,36,65,0,29,55,33,100,44,77,86,73,45,78,0,4
877,25,70,54,100,60,66,35,27,0,1,19,10,61,0,100,0,2
7381,23,78,51,100,100,78,70,40,40,0,78,2,37,37,0,74,8
